In [1]:
# Data ingestion 

from langchain_community.document_loaders import PyPDFLoader

In [2]:
loader = PyPDFLoader("paper.pdf")
docs = loader.load_and_split()
# docs

In [3]:
from dotenv import  load_dotenv
import os 
load_dotenv()

os.environ['jina_api_key'] = os.getenv('jina_api_key')
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')


In [4]:
print(os.getenv('jina_api_key'))
print(os.getenv('GROQ_API_KEY'))

jina_81be7c5b7ed240f994ba3d69590e7b67V1snc1pfP4zhbiLQp35r7NdbkW7K
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


In [5]:
# from langchain.text_splitter import 
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
text_splittter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=10,
)
documents = text_splittter.split_documents(docs)
documents[:5]

[Document(metadata={'source': 'paper.pdf', 'page': 0}, page_content='MDAgents: An Adaptive Collaboration of LLMs for\nMedical Decision-Making\nYubin Kim1 Chanwoo Park1 Hyewon Jeong1♮ Yik Siu Chan1\nXuhai Xu1 Daniel McDuff2 Hyeonhoon Lee3'),
 Document(metadata={'source': 'paper.pdf', 'page': 0}, page_content='Marzyeh Ghassemi1 Cynthia Breazeal1 Hae Won Park1\n1Massachusetts Institute of Technology\n2Google Research\n3Seoul National University Hospital'),
 Document(metadata={'source': 'paper.pdf', 'page': 0}, page_content='{ybkim95,cpark97,hyewonj,yiksiuc,xoxu,mghassem,cynthiab,haewon}@mit.edu\ndmcduff@google.com\nhhoon@snu.ac.kr\nAbstract\nFoundation models are becoming valuable tools in medicine. Yet despite their'),
 Document(metadata={'source': 'paper.pdf', 'page': 0}, page_content='promise, the best way to leverage Large Language Models (LLMs) in complex\nmedical tasks remains an open question. We introduce a novel multi-agent frame-'),
 Document(metadata={'source': 'paper.pdf', 'pa

In [7]:
# from langchain_ollama import OllamaEmbeddings
from langchain_community.embeddings import JinaEmbeddings
from langchain_community.vectorstores import Chroma
# from langchain_community.vectorstores import FAISS

In [8]:
embeddings_jina = JinaEmbeddings(
    jina_api_key=os.getenv('jina_api_key'), model_name="jina-embeddings-v3"
)

In [10]:
embeddings_jina

JinaEmbeddings(session=<requests.sessions.Session object at 0x7b7cd47e7410>, model_name='jina-embeddings-v3', jina_api_key=SecretStr('**********'))

In [11]:
new_dbs = Chroma.from_documents(documents, embedding=embeddings_jina)



AttributeError: `np.float_` was removed in the NumPy 2.0 release. Use `np.float64` instead.

In [56]:
new_dbs

In [75]:
# result= new_dbs.similarity_search("Medical Decision-Making Agents (MDAgents) framework ", k=5)

In [67]:
# result[0].page_content

In [12]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

In [13]:
llm = ChatGroq(temperature=0.6, groq_api_key=os.getenv("GROQ_API_KEY"), model_name="llama-3.3-70b-versatile")


In [14]:
prompt = ChatPromptTemplate.from_template("""
    Answer the following question based only on the provided context.
    Think step by step before providing a detailed answer.
    I will tip you $1000 if the user finds the answer helpful.
    <context>
    {context}
    </context>
    Question: {input}""")

In [15]:
# prompt = ChatPromptTemplate.from_messages([("human", "Answer this topic : {topic}")])
# chain = prompt | llm
# await chain.invoke({"topic": "The Sun"})

In [16]:
from langchain.chains.combine_documents import create_stuff_documents_chain

# document_chain is responsible for put the context and input in the llm model
document_chains = create_stuff_documents_chain(llm, prompt)

In [76]:
# document_chains

In [17]:
# interface to connecting the vector store data
retriever = new_dbs.as_retriever()
retriever
# new_dbs.

NameError: name 'new_dbs' is not defined

In [18]:
from langchain.chains import create_retrieval_chain

response = create_retrieval_chain(retriever, document_chains)


NameError: name 'retriever' is not defined

In [20]:
response_answer = response.invoke({"input": """A 24-year-old woman presents to the emergency department after an episode of altered mental status. She was at a party when she suddenly fell to the floor. Her friends state that she was not responsive and was acting oddly. After the episode, the patient seemed confused and had no memory of the event. The patient has a past medical history of substance abuse, depression, and anxiety. Her temperature is 98.6°F (37.0°C), blood pressure is 117/69 mmHg, pulse is 80/min, respirations are 12/min, and oxygen saturation is 99% on room air. Physical exam reveals a girl who seems confused and responds to questions slowly. Her gait is stable, and she demonstrates 5/5 strength in her upper and lower extremities. Laboratory values and a urine dipstick test are performed as seen below:\n\nToxicology:\nBenzodiazepines: Positive\nMarijuana: Positive\nCocaine: Negative\nPhencyclidine: Negative\n\nUrine Dipstick:\nAppearance: Dark\nRed blood cells: Positive\nGlucose: Negative\n\nWhich of the following is the best next step in management?",

  "options": 
    "A": "Diazepam",
    "B": "Electroencelphalography",
    "C": "IV fluids and mannitol",
    "D": "Phenytoin",
    "E": "Supportive therapy and observation"
    , what would be the correct answer? 
  """})

NameError: name 'response' is not defined

In [77]:
response_answer

{'input': 'Medical Decision-Making Agents (MDAgents) framework',
 'context': [Document(metadata={'page': 2, 'source': 'paper.pdf'}, page_content='task that requires teamwork and should benefit from multi-agent collaboration [72].\n3 MDAgents: Medical Decision-making Agents'),
  Document(metadata={'page': 1, 'source': 'paper.pdf'}, page_content='Inspired by the way that clinicians make decisions in practice, we propose Medical Decision-making\nAgents (MDAgents), an adaptive medical decision-making framework that leverages LLMs to'),
  Document(metadata={'page': 0, 'source': 'MDAgent_Document.pdf'}, page_content='Theprimarygoal of theMDAgentsframeworkistoassistinmedical decision-makingbyprocessingmedical queriesandperformingthefollowingsteps:'),
  Document(metadata={'page': 9, 'source': 'paper.pdf'}, page_content='medical decision processes has been comprehensively evaluated, with MDAgents outperforming\nprevious solo and group methods in 7 out of 10 medical benchmarks. The case study il